In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from scipy import ndimage
import math
import ast #to easily read out class text file that contains some unknwn syntax.
import scipy   #to upscale the image
import matplotlib.pyplot as plt
import cv2     
from keras.applications.resnet import ResNet50, preprocess_input
from keras.models import Model   
from PIL import Image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
(training_images, training_labels) , (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

train_X = preprocess_image_input(training_images)
test_X = preprocess_image_input(test_images)

170498071/170498071 [==============================] - 5s 0us/step


In [4]:
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
 
  return feature_extractor



def classifier(inputs):
    x= tf.keras.layers.Conv2D(2048, (1, 1), strides=(1, 1), padding="same")(inputs)
    x= tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    x = tf.keras.layers.Dense(10,name="classification")(x)
    return x



def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    
    
    classification_output = classifier(resnet_feature_extractor)

    return classification_output




def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


resnet50 = define_compile_model()

resnet50.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2  (None, 224, 224, 3)       0         
 D)                                                              
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 2048)        4196352   
                                                                 
 re_lu (ReLU)                (None, 7, 7, 2048)        0         
                                                                 
 global_average_pooling2d (  (None, 2048)              0      

In [5]:
resnet50.compile(
    optimizer=keras.optimizers.SGD(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [6]:
resnet50.fit(train_X, training_labels, epochs=3, validation_data = (test_X, test_labels), batch_size=64)

Epoch 1/3
782/782 [==============================] - 300s 352ms/step - loss: 0.3994 - accuracy: 0.8683 - val_loss: 0.2175 - val_accuracy: 0.9285
Epoch 2/3
782/782 [==============================] - 272s 348ms/step - loss: 0.1067 - accuracy: 0.9645 - val_loss: 0.1724 - val_accuracy: 0.9411
Epoch 3/3
782/782 [==============================] - 272s 348ms/step - loss: 0.0397 - accuracy: 0.9890 - val_loss: 0.1708 - val_accuracy: 0.9468


In [25]:
#proportionality maintained student
student_bl = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.GlobalAveragePooling2D(),
        layers.Dense(10),
    ],
    name="student_bl",
)

student_bl.summary()

Model: "student_bl"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 16, 16, 128)       3584      
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 16, 16, 128)       0         
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 16, 16, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_29 (Conv2D)          (None, 8, 8, 64)          73792     
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 8, 8, 64)          0         
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 8, 8, 64)          0         
 ng2D)                                                  

In [8]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=2,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 93s 57ms/step - accuracy: 0.4040 - student_loss: 2.3762 - distillation_loss: 6.0535
Epoch 2/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.5167 - student_loss: 1.6170 - distillation_loss: 4.5571
Epoch 3/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.5661 - student_loss: 1.4672 - distillation_loss: 4.1187
Epoch 4/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6001 - student_loss: 1.3370 - distillation_loss: 3.7718
Epoch 5/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6217 - student_loss: 1.2556 - distillation_loss: 3.5563
Epoch 6/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6445 - student_loss: 1.1981 - distillation_loss: 3.3923
Epoch 7/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6534 - student_loss: 1.1545 - distillation_loss: 3.2649
Epoch 8/10
1563/1563

[0.6679999828338623, 1.0420880317687988]

In [24]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 91s 56ms/step - accuracy: 0.4068 - student_loss: 2.2418 - distillation_loss: 9.6235
Epoch 2/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.5278 - student_loss: 1.7327 - distillation_loss: 7.4847
Epoch 3/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.5772 - student_loss: 1.5359 - distillation_loss: 6.7208
Epoch 4/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.6079 - student_loss: 1.4075 - distillation_loss: 6.2179
Epoch 5/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.6308 - student_loss: 1.3231 - distillation_loss: 5.8910
Epoch 6/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6498 - student_loss: 1.2542 - distillation_loss: 5.6376
Epoch 7/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6625 - student_loss: 1.2005 - distillation_loss: 5.4065
Epoch 8/10
1563/1563

[0.671500027179718, 1.1355122327804565]

In [10]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=4,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 91s 56ms/step - accuracy: 0.4124 - student_loss: 2.2044 - distillation_loss: 11.5461
Epoch 2/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.5317 - student_loss: 1.7269 - distillation_loss: 9.1594
Epoch 3/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.5750 - student_loss: 1.5500 - distillation_loss: 8.3771
Epoch 4/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.6053 - student_loss: 1.4230 - distillation_loss: 7.8346
Epoch 5/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6328 - student_loss: 1.3222 - distillation_loss: 7.3896
Epoch 6/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.6459 - student_loss: 1.2624 - distillation_loss: 7.0884
Epoch 7/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.6610 - student_loss: 1.2117 - distillation_loss: 6.8653
Epoch 8/10
1563/156

[0.6491000056266785, 1.3188858032226562]

In [12]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=5,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 91s 56ms/step - accuracy: 0.4069 - student_loss: 2.2010 - distillation_loss: 12.4117
Epoch 2/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.5253 - student_loss: 1.7435 - distillation_loss: 9.9990
Epoch 3/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.5775 - student_loss: 1.5400 - distillation_loss: 9.0779
Epoch 4/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.6092 - student_loss: 1.4032 - distillation_loss: 8.4486
Epoch 5/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.6311 - student_loss: 1.3195 - distillation_loss: 8.0501
Epoch 6/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.6477 - student_loss: 1.2529 - distillation_loss: 7.7201
Epoch 7/10
1563/1563 [==============================] - 88s 56ms/step - accuracy: 0.6617 - student_loss: 1.2004 - distillation_loss: 7.4374
Epoch 8/10
1563/156

[0.6525999903678894, 1.1079679727554321]

In [14]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=6,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 93s 57ms/step - accuracy: 0.4169 - student_loss: 2.1568 - distillation_loss: 12.3398
Epoch 2/10
1563/1563 [==============================] - 90s 58ms/step - accuracy: 0.5330 - student_loss: 1.6872 - distillation_loss: 9.9787
Epoch 3/10
1563/1563 [==============================] - 90s 58ms/step - accuracy: 0.5835 - student_loss: 1.4958 - distillation_loss: 9.1248
Epoch 4/10
1563/1563 [==============================] - 90s 58ms/step - accuracy: 0.6114 - student_loss: 1.3833 - distillation_loss: 8.6097
Epoch 5/10
1563/1563 [==============================] - 90s 58ms/step - accuracy: 0.6322 - student_loss: 1.3019 - distillation_loss: 8.2210
Epoch 6/10
1563/1563 [==============================] - 90s 58ms/step - accuracy: 0.6459 - student_loss: 1.2424 - distillation_loss: 7.8918
Epoch 7/10
1563/1563 [==============================] - 90s 58ms/step - accuracy: 0.6618 - student_loss: 1.1852 - distillation_loss: 7.6287
Epoch 8/10
1563/156

[0.666100025177002, 1.7322781085968018]

In [16]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=7,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 92s 57ms/step - accuracy: 0.4137 - student_loss: 2.1461 - distillation_loss: 12.1106
Epoch 2/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.5265 - student_loss: 1.6790 - distillation_loss: 9.8193
Epoch 3/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.5770 - student_loss: 1.4979 - distillation_loss: 9.0466
Epoch 4/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.6061 - student_loss: 1.3782 - distillation_loss: 8.5298
Epoch 5/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.6251 - student_loss: 1.3056 - distillation_loss: 8.1815
Epoch 6/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6429 - student_loss: 1.2341 - distillation_loss: 7.8474
Epoch 7/10
1563/1563 [==============================] - 89s 57ms/step - accuracy: 0.6550 - student_loss: 1.1953 - distillation_loss: 7.6258
Epoch 8/10
1563/156

[0.6625000238418579, 1.2223527431488037]

In [18]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=8,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 92s 57ms/step - accuracy: 0.4190 - student_loss: 2.0823 - distillation_loss: 11.6396
Epoch 2/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.5303 - student_loss: 1.6585 - distillation_loss: 9.5960
Epoch 3/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.5778 - student_loss: 1.4711 - distillation_loss: 8.8203
Epoch 4/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.6051 - student_loss: 1.3586 - distillation_loss: 8.3212
Epoch 5/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.6277 - student_loss: 1.2819 - distillation_loss: 7.9419
Epoch 6/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.6462 - student_loss: 1.2117 - distillation_loss: 7.6341
Epoch 7/10
1563/1563 [==============================] - 88s 57ms/step - accuracy: 0.6568 - student_loss: 1.1675 - distillation_loss: 7.3947
Epoch 8/10
1563/156

[0.652999997138977, 1.3971123695373535]

In [26]:
class Distiller_bl(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
       
        x, y = data

  
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
           
            student_predictions = self.student(x, training=True)

            
            student_loss = self.student_loss_fn(y, student_predictions)

            
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

       
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        
        self.compiled_metrics.update_state(y, student_predictions)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
       
        x, y = data

      
        y_prediction = self.student(x, training=False)

       
        student_loss = self.student_loss_fn(y, y_prediction)

      
        self.compiled_metrics.update_state(y, y_prediction)

       
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

distiller = Distiller_bl(student=student_bl, teacher=resnet50)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=9,
)


distiller.fit(train_X, training_labels, epochs=10)


distiller.evaluate(test_X, test_labels)



Epoch 1/10
1563/1563 [==============================] - 93s 57ms/step - accuracy: 0.4141 - student_loss: 2.0653 - distillation_loss: 11.2136
Epoch 2/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.5319 - student_loss: 1.6219 - distillation_loss: 9.2394
Epoch 3/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.5812 - student_loss: 1.4391 - distillation_loss: 8.4922
Epoch 4/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.6132 - student_loss: 1.3175 - distillation_loss: 7.9715
Epoch 5/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.6328 - student_loss: 1.2403 - distillation_loss: 7.6298
Epoch 6/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.6483 - student_loss: 1.1825 - distillation_loss: 7.3349
Epoch 7/10
1563/1563 [==============================] - 90s 57ms/step - accuracy: 0.6617 - student_loss: 1.1409 - distillation_loss: 7.1254
Epoch 8/10
1563/156

[0.6535000205039978, 0.8826004266738892]